<a href="https://colab.research.google.com/github/vohoaidanh/AIGCDetectBenchmark/blob/main/AIGCDetectBenchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AIGCDetectBenchmark


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [8]:
!pip install ftfy
!pip install natsort
!pip install tensorboardX
!pip install blobfile
!pip install mpi4py


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 949.3 kB/s eta 0:00:00


In [2]:
!git clone https://github.com/vohoaidanh/AIGCDetectBenchmark.git

Cloning into 'AIGCDetectBenchmark'...
remote: Enumerating objects: 430, done.
remote: Counting objects: 100% (430/430), done.
remote: Compressing objects: 100% (293/293), done.
remote: Total 430 (delta 157), reused 398 (delta 125), pack-reused 0
Receiving objects: 100% (430/430), 3.97 MiB | 19.25 MiB/s, done.
Resolving deltas: 100% (157/157), done.


In [ ]:
 file_path = '/content/drive/MyDrive/DATASETS/real_gen_dataset.zip'
 !mkdir -p /content/AIGCDetectBenchmark/dataset
 !unzip "$file_path" -d /content/AIGCDetectBenchmark/dataset

In [6]:
!cp '/content/drive/MyDrive/WEIGHTS/sidd_rgb.pth' /content/AIGCDetectBenchmark/weights/preprocessing

In [1]:
%cd /content/AIGCDetectBenchmark

/content/AIGCDetectBenchmark


In [ ]:
#!python test_sidd_rgb_test.py --input_dir dataset/real_gen_dataset --result_dir dataset/data

In [20]:
!python train.py --name test_v1 --dataroot dataset/data/test --detect_method CNNSpot --blur_prob 0.1 --blur_sig 0.0,3.0 --jpg_prob 0.1 --jpg_method cv2,pil --jpg_qual 30,100

----------------- Options ---------------
                 CropSize: 224                           
               batch_size: 64                            
                    beta1: 0.9                           
                blur_prob: 0.1                           
                 blur_sig: 0.0,3.0                       
          checkpoints_dir: ./checkpoints                 
                  classes: ['airplane', 'bird', 'bicycle', 'boat', 'bottle', 'bus', 'car', 'cat', 'cow', 'chair', 'diningtable', 'dog', 'person', 'pottedplant', 'motorbike', 'tvmonitor', 'train', 'sheep', 'sofa', 'horse']	[default: airplane,bird,bicycle,boat,bottle,bus,car,cat,cow,chair,diningtable,dog,person,pottedplant,motorbike,tvmonitor,train,sheep,sofa,horse]
           continue_train: False                         
                 data_aug: False                         
                 dataroot: dataset/data/test             	[default: /hotdata/share/AIGCDetect]
            detect_method: CNNSp